### Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils import to_categorical
from tools import *

Using TensorFlow backend.


In [2]:
!pip install -r requirements.txt

In [21]:
BATCH_SIZE = 256
BUFFER_SIZE = 100000
EPOCHS = 1
OUTPUT_DIR = "img"


### Loading Data

In [4]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

labels = get_label_dict()
words = get_word_labels()

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data

Loading labels

Done

Loading words

Done



In [22]:
X_train.shape

(100000, 64, 64, 3)

### Models

In [40]:
class Generator(keras.Model):
    
    def __init__(self, random_noise_size = 100):
        super().__init__(name='generator')
        #layers
        self.input_layer = keras.layers.Dense(units = random_noise_size)
        self.dense_1 = keras.layers.Dense(units = 128)
        self.leaky_1 =  keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_2 = keras.layers.Dense(units = 128)
        self.leaky_2 = keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_3 = keras.layers.Dense(units = 256)
        self.leaky_3 = keras.layers.LeakyReLU(alpha = 0.01)
        self.output_layer = keras.layers.Dense(units=3, activation = "tanh")
        
    def call(self, input_tensor):
        ## Definition of Forward Pass
        x = self.input_layer(input_tensor)
        x = self.dense_1(x)
        x = self.leaky_1(x)
        x = self.dense_2(x)
        x = self.leaky_2(x)
        x = self.dense_3(x)
        x = self.leaky_3(x)
        return  self.output_layer(x)
    
    def generate_noise(self,batch_size, random_noise_size):
        return np.random.uniform(-1,1, size = (batch_size, random_noise_size, 3))


In [72]:
class Discriminator(keras.Model):
    def __init__(self):
        super().__init__(name = "discriminator")
        
        #Layers
        self.input_layer = keras.layers.Dense(units = 1, input_shape=(64, 64, 3))
        self.dense_1 = keras.layers.Dense(units = 128)
        self.leaky_1 =  keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_2 = keras.layers.Dense(units = 128)
        self.leaky_2 = keras.layers.LeakyReLU(alpha = 0.01)
        self.dense_3 = keras.layers.Dense(units = 128)
        self.leaky_3 = keras.layers.LeakyReLU(alpha = 0.01)
        
        self.logits = keras.layers.Dense(units = 1)  # This neuron tells us if the input is fake or real
    def call(self, input_tensor):
          ## Definition of Forward Pass
        x = self.input_layer(input_tensor)
        x = self.dense_1(x)
        x = self.leaky_1(x)
        x = self.leaky_2(x)
        x = self.leaky_3(x)
        x = self.leaky_3(x)
        x = self.logits(x)
        return x

In [73]:
discriminator  = Discriminator()
generator = Generator()

### Objective

In [74]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)


In [75]:
def generator_objective(dx_of_gx):
    # Labels are true here because generator thinks he produces real images. 
    
    return cross_entropy(tf.ones_like(dx_of_gx), dx_of_gx)

In [109]:
def discriminator_objective(real_output, fake_output, smoothing_factor=0.9):
    """
    label = 0: fake
    label = 1: real
    """
    real_loss = cross_entropy(tf.ones_like(real_output * smoothing_factor, real_output.ref()))
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    return real_loss + fake_loss

In [110]:
generator_optimizer = keras.optimizers.RMSprop()
discriminator_optimizer = keras.optimizers.RMSprop()

In [111]:
@tf.function()
def training_step(generator: Generator, discriminator: Discriminator, images:np.ndarray , k:int=1, batch_size=32):
    for _ in range(k):
         with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            noise = generator.generate_noise(batch_size, 64)
#             print(g_z.shape)
            g_z = generator(noise)

            images = tf.cast(images, tf.float32)
            d_x_true = discriminator(images) # Trainable?
            
            d_x_fake = discriminator(g_z) # dx_of_gx

            
            discriminator_loss = discriminator_objective(d_x_true, d_x_fake)
            # Adjusting Gradient of Discriminator
            disc_grad = disc_tape.gradient(discriminator_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(disc_grad, discriminator.trainable_variables)) # Takes a list of gradient and variables pairs
            
              
            generator_loss = generator_objective(d_x_fake)
            # Adjusting Gradient of Generator
            gen_grad = gen_tape.gradient(generator_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gen_grad, generator.trainable_variables))

In [112]:
seed = np.random.uniform(-1,1, size = (1, 100)) # generating some noise for the training


In [113]:
def training(dataset, epoches):
    for epoch in range(epoches):
        for batch in dataset: 
#             print(type(batch))
#             print(type(epoches))
#             print(type(BATCH_SIZE))
#             print(type(generator))
#             print(type(discriminator))
            training_step(generator, discriminator, batch, k=1, batch_size=BATCH_SIZE)
            
        ## After ith epoch plot image 
        if (epoch % 50) == 0: 
            fake_image = tf.reshape(generator(seed), shape=(28,28))
            print("{}/{} epoches".format(epoch, epoches))
            #plt.imshow(fake_image, cmap = "gray")
            plt.imsave("{}/{}.png".format(OUTPUT_DIR,epoch),fake_image, cmap = "gray")

In [114]:
%%time
training(X_train, EPOCHS)

TypeError: in user code:

    <ipython-input-111-c96fecbe8241>:15 training_step  *
        discriminator_loss = discriminator_objective(d_x_true, d_x_fake)
    <ipython-input-109-98d251d86973>:6 discriminator_objective  *
        real_loss = cross_entropy(tf.ones_like(real_output * smoothing_factor, real_output.ref()))
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:180 wrapper  **
        return target(*args, **kwargs)
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:2918 ones_like_v2
        return ones_like_impl(input, dtype, name, optimize=True)
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:2928 ones_like_impl
        ret = ones(ones_shape, dtype=dtype, name=name)
    /Users/sofiapaganin/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:2959 ones
        dtype = dtypes.as_dtype(dtype).base_dtype
    /Users/sofiapa

In [108]:
tf.__version__

'2.2.0-rc4'

In [16]:
keras.__version__

'2.3.0-tf'

### Sample Outputs

In [17]:
fake_image = generator(np.random.uniform(-1,1, size = (1, 100)))
plt.imshow(tf.reshape(fake_image, shape = (28,28)), cmap="gray")

InvalidArgumentError: Matrix size-incompatible: In[0]: [1,100], In[1]: [3,100] [Op:MatMul]